# Gallery Example: M/Par/1 Queue

This example demonstrates an M/Pareto/1 queueing system:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Pareto distribution (heavy-tailed)
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

Pareto service times create heavy-tailed behavior with occasional very long service times.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mpar1(): Network {
    """Create M/Par/1 queueing model"""
    val model = Network("M/Par/1")
// Block 1: nodes
    val source = Source(model, "Source")
    val queue = Queue(model, "Queue", SchedStrategy.FCFS)
    val sink = Sink(model, "Sink")
// Block 2: classes
    val oclass1 = OpenClass(model, "Class1")
    source.setArrival(oclass1, Exp(1))
// Pareto service with mean=0.5 and SCV=64 (very heavy tail)
    queue.setService(oclass1, Pareto.fit_mean_and_scv(0.5, 64))
// Block 3: topology
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addRoute(oclass1, queue, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_mpar1()

## About Pareto Service Times

The Pareto distribution:
- Is a power-law distribution with heavy tail
- Commonly used for modeling file sizes, network traffic, etc.
- Can have extremely high variability (SCV >> 1)

With SCV = 64:
- Variance is 64 times the squared mean
- Occasional very long service times occur
- Creates significant queue buildup during long services

This model demonstrates the impact of heavy-tailed service on queueing performance.

In [ ]:
// Solve with Fluid solver (CTMC may struggle with heavy tails)
println("\n=== Solver Results ===")
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)